In [49]:
import seaborn as sns
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
import itertools
from natsort import natsorted, natsort_keygen


In [50]:
resources = Path(os.path.abspath('../resources'))
all_test_smell_path = Path(os.path.abspath('../resources/tsdetect/all_test_smell'))
test_smell_hive_path = Path(os.path.abspath('../resources/tsdetect/test_smell_hive'))


In [51]:
opened_df = pd.read_csv(all_test_smell_path/'all_opened.csv', header=None).sort_values(by=[0], key=natsort_keygen()).reset_index(drop=True)
closed_df = pd.read_csv(all_test_smell_path/'all_closed.csv',header=None).sort_values(by=[0], key=natsort_keygen()).reset_index(drop=True)

opened_df.to_pickle(all_test_smell_path/"opened_df.pkl")
closed_df.to_pickle(all_test_smell_path/"closed_df.pkl")


In [52]:
test_semantic_smell = ['Assertion Roulette','Conditional Test Logic','Duplicate Assert']
issue_in_test_step = ['Exception Catching Throwing', 'General Fixture', 'EmptyTest', 'Redundant Assertion', 'Unknown Test', 'Constructor Initialization']
code_related = ['Magic Number Test', 'Print Statement', 'IgnoredTest', 'Verbose Test']
dependencies = ['Mystery Guest', 'Resource Optimism']
test_execution = ['Sensitive Equality', 'Sleepy Test', 'Sensitive Equality']

Testsmell_list = [test_semantic_smell, issue_in_test_step, code_related, dependencies, test_execution]
Testsmell_list_string = ["test_semantic_smell", "issue_in_test_step", "code_related", "dependencies", "test_execution"]

# for index, sublist_test_smell in enumerate(Testsmell_list):
#     test_smell_category = Testsmell_list_string[index]
#     for a_test_smell in sublist_test_smell:
#         # Do something with each item in the sublist
#         print(f"Processing item '{a_test_smell}' from {test_smell_category}")

In [106]:
def clean_data(df_input: pd, wanted_test_smell: list, classification_test_smell: str):
    result_df = pd.DataFrame(columns=['Test_Smell_Classification_Type', 'Test_Smell', 'Number_Of_Test_Class','Total_Of_Test_Class', 'List_Of_Test_Class',  'Number_Of_Test_Smell', 'Total_Of_Test_Smell'])
    df_input = df_input.dropna(axis='columns')
    test_smell_df = df_input.loc[:, 'Assertion Roulette':'Dependent Test']
    test_smell_df['TestClass'] = df_input['TestClass']

    for column in wanted_test_smell:
        test_smell = pd.DataFrame({'Test_Smell': [column]})
        temp_test_class = test_smell_df.loc[test_smell_df[column] > 0, 'TestClass'].tolist()
        list_of_test_class = pd.DataFrame({'List_Of_Test_Class': [temp_test_class]})
        number_of_test_class = pd.DataFrame({'Number_Of_Test_Class': [len(temp_test_class)]})
        Number_Of_Test_Smell = pd.DataFrame({'Number_Of_Test_Smell': [test_smell_df[column].sum()]})
        temp_list = pd.concat([test_smell, list_of_test_class, number_of_test_class, Number_Of_Test_Smell], axis=1)
        result_df = pd.concat([result_df,temp_list])

    result_df['Total_Of_Test_Class'] = len(list(dict.fromkeys(list(itertools.chain.from_iterable(result_df['List_Of_Test_Class'].tolist())))))
    result_df['Test_Smell_Classification_Type'] = classification_test_smell
    result_df['Total_Of_Test_Smell'] = result_df['Number_Of_Test_Smell'].sum()

    return result_df

demodf = pd.read_csv("/Users/Jumma/github_repo/github_api_extractor/resources/tsdetect/test_smell_hive/Output_TestSmellDetection_closed_hive_file_0_f205a94c39b612a22a1e0bca1494e06ce52df2ce.csv")

result = clean_data(demodf, test_semantic_smell, "test_semantic_smell")
result

,Test_Smell_Classification_Type,Test_Smell,Number_Of_Test_Class,Total_Of_Test_Class,List_Of_Test_Class,Number_Of_Test_Smell,Total_Of_Test_Smell
0,test_semantic_smell,Assertion Roulette,482,533,"[TestHiveDruidQueryBasedInputFormat.java, Test...",2220,3866
0,test_semantic_smell,Conditional Test Logic,244,533,"[TestPrimitiveComparisonFilter.java, TestAccum...",827,3866
0,test_semantic_smell,Duplicate Assert,250,533,"[TestDruidStorageHandler.java, TestAccumuloPre...",819,3866


In [54]:
def read_csv(csv):
    df = pd.read_csv(csv)
    return df
opened_pickle = all_test_smell_path/"opened_df.pkl"
closed_pickle = all_test_smell_path/"closed_df.pkl"
open_path = all_test_smell_path/"opened/"
close_path = all_test_smell_path/"closed/"

def do_something(pickle, save):
    filename = os.path.basename(pickle).split('.')[0]
    df = pd.read_pickle(pickle)
    for index, row in df.iterrows():
        file_path_to_output_test_smell = str(row.iloc[0])
        print(f"iteration at {index} read file : {test_smell_hive_path}/{file_path_to_output_test_smell}")
        df_temp = read_csv(test_smell_hive_path/file_path_to_output_test_smell)
        temp = []
        for round, sublist_test_smell in enumerate(Testsmell_list):
            test_smell_category = Testsmell_list_string[round]
            print(f"test smell category at {test_smell_category} count round {round}")
            one_of_test_smell = clean_data(df_temp, sublist_test_smell, test_smell_category)
            temp.append(one_of_test_smell)
        print(f'end testsmelllist, by have {len(temp)} in temp list')
        result = pd.concat(temp).reset_index(drop=True)
        del temp
        result.to_pickle(save/f"from_{filename}_{file_path_to_output_test_smell}.pkl")
    return result
            # for a_test_smell in sublist_test_smell:
            #     # Do something with each item in the sublist
            #     print(f"Processing item '{a_test_smell}' from {test_smell_category}")
do_something(opened_pickle, open_path)
do_something(closed_pickle, close_path)

iteration at 0 read file : /Users/Jumma/github_repo/github_api_extractor/resources/tsdetect/test_smell_hive/Output_TestSmellDetection_opened_hive_file_0_422fd4e48bdd21bbb0597be03b702cc410673e66.csv
test smell category at test_semantic_smell count round 0
test smell category at issue_in_test_step count round 1
test smell category at code_related count round 2
test smell category at dependencies count round 3
test smell category at test_execution count round 4
end testsmelllist, by have 5 in temp list
iteration at 1 read file : /Users/Jumma/github_repo/github_api_extractor/resources/tsdetect/test_smell_hive/Output_TestSmellDetection_opened_hive_file_1_29dc08172ba9ae3a3f51320f656994c20198c5f0.csv
test smell category at test_semantic_smell count round 0
test smell category at issue_in_test_step count round 1
test smell category at code_related count round 2
test smell category at dependencies count round 3
test smell category at test_execution count round 4
end testsmelllist, by have 5 in 

,Test_Smell_Classification_Type,Test_Smell,Number_Of_Test_Class,Total_Of_Test_Class,List_Of_Test_Class,Number_Of_Test_Smell,Total_Of_Test_Smell
0,test_semantic_smell,Assertion Roulette,368,408,"[TestHiveDruidQueryBasedInputFormat.java, Test...",1801,3134
1,test_semantic_smell,Conditional Test Logic,203,408,"[TestPrimitiveComparisonFilter.java, TestAccum...",713,3134
2,test_semantic_smell,Duplicate Assert,194,408,"[TestHiveDruidQueryBasedInputFormat.java, Test...",620,3134
3,issue_in_test_step,Exception Catching Throwing,486,498,"[TestHiveDruidQueryBasedInputFormat.java, Test...",3177,5218
4,issue_in_test_step,General Fixture,98,498,"[TestAccumuloPredicateHandler.java, TestHiveAc...",845,5218
5,issue_in_test_step,EmptyTest,4,498,"[TestDummy.java, TestDummy.java, TestTempleton...",4,5218
6,issue_in_test_step,Redundant Assertion,18,498,"[TestHiveMetaStore.java, TestHCatClient.java, ...",139,5218
7,issue_in_test_step,Unknown Test,207,498,"[TestDruidSerDe.java, TestHiveAccumuloTableInp...",995,5218
8,issue_in_test_step,Constructor Initialization,58,498,"[TestHiveMetaStoreTxns.java, cbo_rp_TestJdbcDr...",58,5218
9,code_related,Magic Number Test,566,565,"[TestHiveDruidQueryBasedInputFormat.java, Test...",3901,4025


In [61]:
selected_columns = ['Test_Smell_Classification_Type', 'Number_Of_Test_Smell', 'Total_Of_Test_Smell']

file_in_opened = os.listdir(open_path)
file_in_opened = natsorted(file_in_opened)
file_in_opened_path = [os.path.join(open_path, file_name) for file_name in file_in_opened if file_name != ".DS_Store"]
file_in_closed = os.listdir(close_path)
file_in_closed = natsorted(file_in_closed)
file_in_closed_path = [os.path.join(close_path, file_name) for file_name  in file_in_closed if file_name != ".DS_Store"]
print(file_in_opened_path[0])

import glob
hive_url_open_close = Path(os.path.abspath('../resources/hive_use_for_run_java.pkl'))
hive_url_open_close = pd.read_pickle(hive_url_open_close)

def filter_table(pickle):
    df = pd.read_pickle(pickle)
    df = df.loc[:, selected_columns]
    return df

def merge_data(file_in_opened, file_in_closed):
    temp = []
    for index, url in hive_url_open_close.iterrows():
        open_version = filter_table(file_in_opened[index])
        close_version = filter_table(file_in_closed[index])
        open_version.insert(0, 'url', url[0], allow_duplicates=True)
        open_version.insert(1, 'sha_opened', url[1], allow_duplicates=True)
        close_version.insert(0, 'url', url[0], allow_duplicates=True)
        close_version.insert(1, 'sha_closed', url[2], allow_duplicates=True)
        df_concatenated = pd.concat([open_version, close_version]).reset_index(drop=True)
        temp.append(df_concatenated)
    result = pd.concat(temp).reset_index(drop=True)
    result.to_pickle(all_test_smell_path/"merge_data.pkl")
    result.to_csv(all_test_smell_path/"merge_data.csv")
    return result

merge_data(file_in_opened_path, file_in_closed_path)



/Users/Jumma/github_repo/github_api_extractor/resources/tsdetect/all_test_smell/opened/from_opened_df_Output_TestSmellDetection_opened_hive_file_0_422fd4e48bdd21bbb0597be03b702cc410673e66.csv.pkl


,url,sha_opened,Test_Smell_Classification_Type,Number_Of_Test_Smell,Total_Of_Test_Smell,sha_closed
0,https://api.github.com/repos/apache/hive/pulls...,422fd4e48bdd21bbb0597be03b702cc410673e66,test_semantic_smell,2220,3866,NaN
1,https://api.github.com/repos/apache/hive/pulls...,422fd4e48bdd21bbb0597be03b702cc410673e66,test_semantic_smell,827,3866,NaN
2,https://api.github.com/repos/apache/hive/pulls...,422fd4e48bdd21bbb0597be03b702cc410673e66,test_semantic_smell,819,3866,NaN
3,https://api.github.com/repos/apache/hive/pulls...,422fd4e48bdd21bbb0597be03b702cc410673e66,issue_in_test_step,4943,7748,NaN
4,https://api.github.com/repos/apache/hive/pulls...,422fd4e48bdd21bbb0597be03b702cc410673e66,issue_in_test_step,1194,7748,NaN
...,...,...,...,...,...,...
42619,https://api.github.com/repos/apache/hive/pulls/98,NaN,dependencies,45,101,58d1befa2131254b53122b3573189ac1c5022217
42620,https://api.github.com/repos/apache/hive/pulls/98,NaN,dependencies,56,101,58d1befa2131254b53122b3573189ac1c5022217
42621,https://api.github.com/repos/apache/hive/pulls/98,NaN,test_execution,371,766,58d1befa2131254b53122b3573189ac1c5022217
42622,https://api.github.com/repos/apache/hive/pulls/98,NaN,test_execution,24,766,58d1befa2131254b53122b3573189ac1c5022217


In [89]:
import pandas as pd

def compare_total_test_smell(df):
    # Group the DataFrame by the "url" column
    grouped = df.groupby("url")

    # Initialize an empty list to store the comparison results
    results = []

    # Iterate over the grouped data
    for _, group in grouped:
        # Get the "Total_Of_Test_Smell" values for sha_opened and sha_closed
        total_test_smell_opened = group[group["sha_opened"].notnull()]["Total_Of_Test_Smell"].values
        total_test_smell_closed = group[group["sha_closed"].notnull()]["Total_Of_Test_Smell"].values

        # Compare the values and assign 1 or 0
        if len(total_test_smell_opened) > 0 and len(total_test_smell_closed) > 0:
            result = total_test_smell_opened >= total_test_smell_closed
        else:
            result = None

        # Append the result to the list
        results.append(result)

    # Create a dictionary with unique "url" values as keys and the corresponding comparison result as values
    result_dict = dict(zip(grouped.groups.keys(), results))

    # Add a new column "Comparison_Result" with the comparison results in the DataFrame
    df["Comparison_Result"] = df["url"].map(result_dict)

    return df

data = pd.read_pickle(all_test_smell_path/"merge_data.pkl")  # Read the data from the CSV file
result = compare_total_test_smell(data)
result


,url,sha_opened,Test_Smell_Classification_Type,Number_Of_Test_Smell,Total_Of_Test_Smell,sha_closed,Comparison_Result
0,https://api.github.com/repos/apache/hive/pulls...,422fd4e48bdd21bbb0597be03b702cc410673e66,test_semantic_smell,2220,3866,NaN,"[True, True, True, True, True, True, True, Tru..."
1,https://api.github.com/repos/apache/hive/pulls...,422fd4e48bdd21bbb0597be03b702cc410673e66,test_semantic_smell,827,3866,NaN,"[True, True, True, True, True, True, True, Tru..."
2,https://api.github.com/repos/apache/hive/pulls...,422fd4e48bdd21bbb0597be03b702cc410673e66,test_semantic_smell,819,3866,NaN,"[True, True, True, True, True, True, True, Tru..."
3,https://api.github.com/repos/apache/hive/pulls...,422fd4e48bdd21bbb0597be03b702cc410673e66,issue_in_test_step,4943,7748,NaN,"[True, True, True, True, True, True, True, Tru..."
4,https://api.github.com/repos/apache/hive/pulls...,422fd4e48bdd21bbb0597be03b702cc410673e66,issue_in_test_step,1194,7748,NaN,"[True, True, True, True, True, True, True, Tru..."
...,...,...,...,...,...,...,...
42619,https://api.github.com/repos/apache/hive/pulls/98,NaN,dependencies,45,101,58d1befa2131254b53122b3573189ac1c5022217,"[False, False, False, False, False, False, Fal..."
42620,https://api.github.com/repos/apache/hive/pulls/98,NaN,dependencies,56,101,58d1befa2131254b53122b3573189ac1c5022217,"[False, False, False, False, False, False, Fal..."
42621,https://api.github.com/repos/apache/hive/pulls/98,NaN,test_execution,371,766,58d1befa2131254b53122b3573189ac1c5022217,"[False, False, False, False, False, False, Fal..."
42622,https://api.github.com/repos/apache/hive/pulls/98,NaN,test_execution,24,766,58d1befa2131254b53122b3573189ac1c5022217,"[False, False, False, False, False, False, Fal..."


In [105]:
import pandas as pd

def compare_total_test_smell(df):
    result_temp = []

    # Iterate over the unique URLs in the DataFrame
    for url in df["url"].unique():
        # Filter the data for the current URL
        url_data = df[df["url"] == url]

        # Get the total test smell values for opened and closed SHA
        opened_total = url_data[url_data["sha_opened"].notnull()]["Total_Of_Test_Smell"].values
        closed_total = url_data[url_data["sha_closed"].notnull()]["Total_Of_Test_Smell"].values
        test_smell_type = url_data[url_data["sha_closed"].notnull()]["Test_Smell_Classification_Type"].values
        # print(test_smell_type)

        # Compare the values and assign 1 or 0
        if len(opened_total) > 0 and len(closed_total) > 0:
            result = opened_total < closed_total
        else:
            result = None
        results_df = pd.DataFrame({"url": url,"test_smell_type": test_smell_type ,"opened_total":opened_total, "closed_total":closed_total , "Comparison_Result": result})
        result_temp.append(results_df)
        # Append the result to the results DataFrame

        comparison_df = pd.concat(result_temp).reset_index(drop=True)
        comparison_df.to_pickle(all_test_smell_path/"x.pkl")
        comparison_df.to_csv(all_test_smell_path/"x.csv")
    return comparison_df


data = pd.read_pickle(all_test_smell_path/"merge_data.pkl")  # Read the data from the CSV file
result = compare_total_test_smell(data)
result


,url,test_smell_type,opened_total,closed_total,Comparison_Result
0,https://api.github.com/repos/apache/hive/pulls...,test_semantic_smell,3866,3866,False
1,https://api.github.com/repos/apache/hive/pulls...,test_semantic_smell,3866,3866,False
2,https://api.github.com/repos/apache/hive/pulls...,test_semantic_smell,3866,3866,False
3,https://api.github.com/repos/apache/hive/pulls...,issue_in_test_step,7748,7748,False
4,https://api.github.com/repos/apache/hive/pulls...,issue_in_test_step,7748,7748,False
...,...,...,...,...,...
21307,https://api.github.com/repos/apache/hive/pulls/98,dependencies,101,101,False
21308,https://api.github.com/repos/apache/hive/pulls/98,dependencies,101,101,False
21309,https://api.github.com/repos/apache/hive/pulls/98,test_execution,766,766,False
21310,https://api.github.com/repos/apache/hive/pulls/98,test_execution,766,766,False
